# Tractor Routing

NOTE: This notebook is meant to be run in an environment where Optapy has been installed. Although it's a Python module, Optapy requires Java to function.

## Goal

Using a fleet of tractors, treat each field and get back to the barn. Each tractor can treat multiple fields, but has a limited capacity.

## Set the Problem Facts

Obviously, problem facts are facts you know about the problem. As such, they do not change during solving (and thus cannot have any planning variables). For Tractor Routing, the problem facts are the locations a tractor can visit: they are the barns and the fields to treat.

- Locations are coordinates of objects. They also have a special property, a map of the distances they are from every other locations. You can think of it as a big two-entries distance table.

In [ ]:
from optapy import problem_fact


@problem_fact
class Location:
    def __init__(self, X, Y, distance_map=None):
        self.X = X
        self.Y = Y
        self.distance_map = distance_map

    def set_distance_map(self, distance_map):
        self.distance_map = distance_map

    def get_distance_to(self, location):
        return self.distance_map[location]

    def to_X_Y_tuple(self):
        return (
            self.X,
            self.Y
        )

    def __str__(self):
        return f'[{self.X}, {self.Y}]'


- The distance_map is populated by a distance calculator.

In [ ]:
import math


@problem_fact
class EuclideanDistanceCalculator:
    def __init__(self):
        pass

    def calculate_distance(self, start, end):
        if start == end:
            return 0
        X_diff = end.X - start.X
        Y_diff = end.Y - start.Y
        return math.ceil(math.sqrt(X_diff**2 + Y_diff**2))

    def init_distance_maps(self, location_list):
        for location in location_list:
            distance_map = dict()
            for other_location in location_list:
                distance_map[other_location] = self.calculate_distance(location, other_location)
            location.set_distance_map(distance_map)

- A tractor departs and comes back to a barn to reload. Each barn has a name and a location.

In [ ]:
@problem_fact
class Barn:
    def __init__(self, name, location):
        self.name = name
        self.location = location

    def __str__(self):
        return f'BarnLocation {self.name}'

- A tractor visits fields to treat crops. Each field has a name (an id), and a "demand" that takes up some capacity of the tractor.

In [ ]:
@problem_fact
class Field:
    def __init__(self, name, location, demand):
        self.name = name
        self.location = location
        self.demand = demand

    def __str__(self):
        return f'Field {self.name}'

## Set the planning Entities

In Tractor Routing, tractors are the planning entities. Each tractor has a fixed barn to depart from and return to, and a list of fields to visit which we want to plan. In particular:

- A field will be visited by exactly one tractor
- The order of the fields in the list is significant

as such, the field list can be modelled as a planning list variable.

In [ ]:
from optapy import planning_entity, planning_list_variable


@planning_entity
class Tractor:
    def __init__(self, name, capacity, barn, field_list=None):
        self.name = name
        self.capacity = capacity
        self.barn = barn
        if field_list is None:
            self.field_list = []
        else:
            self.field_list = field_list

    @planning_list_variable(Field, ['field_range'])
    def get_field_list(self):
        return self.field_list

    def set_field_list(self, field_list):
        self.field_list = field_list

    def get_route(self):
        if len(self.field_list) == 0:
            return []
        route = [self.barn.location]
        for field in self.field_list:
            route.append(field.location)
        route.append(self.barn.location)
        return route

    def __str__(self):
        return f'Tractor {self.name}'

The `@planning_list_variable` decorator tells OptaPy the method is a getter for a planning list variable. It takes two parameters:

- The first parameter is the type this planning variable takes.
- The second parameter, `value_range_provider_refs`, describes where it gets its values from.
  It a list of the id of its value range providers. This is defined later in the `@planning_solution` class.


## Set the Constraints

- In tractor routing, we have one hard constraint: no tractor can go over its capacity

In [ ]:
from optapy.score import HardSoftScore


def get_total_demand(tractor):
    total_demand = 0
    for field in tractor.field_list:
        total_demand += field.demand
    return total_demand


def tractor_capacity(constraint_factory):
    return constraint_factory \
        .for_each(Tractor) \
        .filter(lambda tractor: get_total_demand(tractor) > tractor.capacity) \
        .penalize("Over tractor capacity", HardSoftScore.ONE_HARD,
                  lambda tractor: int(get_total_demand(tractor) - tractor.capacity))

The lambda in penalize controls how much to penalize for a violation. We want a tractor 5 over capacity to be penalized more than a tractor only 1 over capacity. Hence we penalize by

`get_total_demand(tractor) - tractor.capacity`

which is how much over capacity the tractor is.

- We also have one soft constraint: minimize the total distance

In [ ]:
def get_total_distance(tractor):
    total_distance = 1
    last_location = tractor.barn.location
    for field in tractor.field_list:
        total_distance += field.location.get_distance_to(last_location)
        last_location = field.location
    if last_location is not tractor.barn.location:
        total_distance += tractor.barn.location.get_distance_to(last_location)
    return total_distance


def total_distance(constraint_factory):
    return constraint_factory \
        .for_each(Tractor) \
        .penalize("Minimize total distance", HardSoftScore.ONE_SOFT,
                  lambda tractor: int(get_total_distance(tractor)))

Every initialized Tractor is penalized in the constraint, since we want to minimalize total distance.

- Return a list containing the constraints in a `@constraint_provider` decorated function:

In [ ]:
from optapy import constraint_provider


@constraint_provider
def tractor_routing_constraints(constraint_factory):
    return [
        # Hard constraints
        tractor_capacity(constraint_factory),

        # Soft constraints
        total_distance(constraint_factory)
    ]

## Planning Solution

Finally, there is the planning solution. The planning solution stores references to all the problem facts and planning entities that define the problem. Additionally, it also contain the score of the solution. The planning solution class represent both the problem and the solution; as such, a problem can be viewed as an uninitialized planning solution.

In Tractor Routing, it needs to contain the field list, tractor list, and the score. The bounds are included for easier visualization of the route:

In [ ]:
from optapy import planning_solution, planning_entity_collection_property, \
    problem_fact_collection_property, value_range_provider, planning_score


@planning_solution
class TractorRoutingSolution:
    def __init__(self,  name, location_list, barn_list, tractor_list, field_list,
                 south_west_corner, north_east_corner, score=None):
        self.name = name
        self.location_list = location_list
        self.barn_list = barn_list
        self.tractor_list = tractor_list
        self.field_list = field_list
        self.south_west_corner = south_west_corner
        self.north_east_corner = north_east_corner
        self.score = score

    @planning_entity_collection_property(Tractor)
    def get_tractor_list(self):
        return self.tractor_list

    @problem_fact_collection_property(Field)
    @value_range_provider('field_range', value_range_type=list)
    def get_field_list(self):
        return self.field_list

    @problem_fact_collection_property(Location)
    def get_location_list(self):
        return self.location_list

    @problem_fact_collection_property(Barn)
    def get_barn_list(self):
        return self.barn_list

    @planning_score(HardSoftScore)
    def get_score(self):
        return self.score

    def set_score(self, score):
        self.score = score

    def get_bounds(self):
        return [self.south_west_corner.to_X_Y_tuple(), self.north_east_corner.to_X_Y_tuple()]

    def get_distance_meters(self):
        return -self.score.getSoftScore() if self.score is not None else 0

## Solving

Now that we defined our model and constraints, let's create an instance of the problem. We will use the following parameters:
- 15 tractors
- Max capacity of each tractor is 5
- 3 different barns (tractors randomly assigned)
- 50 fields to treat

In [ ]:
from data import DataBuilder

problem = DataBuilder.builder(Location, Barn, Field, Tractor, TractorRoutingSolution, 
                              EuclideanDistanceCalculator()) \
                             .set_tractor_capacity(5) \
                             .set_field_count(50).set_tractor_count(15).set_barn_count(3) \
                             .set_south_west_corner(Location(0, 1079)) \
                             .set_north_east_corner(Location(1919, 0)).build()

and solve it:

In [ ]:
from optapy import solver_manager_create
import optapy.config
from optapy.types import Duration

SINGLETON_ID = 1
solver_config = optapy.config.solver.SolverConfig()
solver_config \
    .withSolutionClass(TractorRoutingSolution) \
    .withEntityClasses(Tractor) \
    .withConstraintProviderClass(tractor_routing_constraints) \
    .withTerminationSpentLimit(Duration.ofSeconds(10))

solver_manager = solver_manager_create(solver_config)
last_score = HardSoftScore.ZERO

tractor_routing_solution = problem

best_solution = solver_manager.solve(SINGLETON_ID, lambda _: problem)
final_solution = best_solution.getFinalBestSolution()

## We can now plot the solution!

In [ ]:
from matplotlib import pyplot as plt, patches
from matplotlib.path import Path

%matplotlib inline
color_index = 0


def generate_color():
    global color_index
    colors =  (
      'aqua',
      'aquamarine',
      'blue',
      'blueviolet',
      'chocolate',
      'cornflowerblue',
      'crimson',
      'forestgreen',
      'gold',
      'lawngreen',
      'limegreen',
      'maroon',
      'mediumvioletred',
      'orange',
      'slateblue',
      'tomato'
    )
    out = colors[color_index]
    color_index = (color_index + 1) % len(colors)
    return out


plt.rcParams["figure.autolayout"] = True
fig, ax = plt.subplots(1, 1)

tractor_to_route_dict = dict()
tractor_to_route_color_dict = dict()
field_to_marker_dict = dict()

for field in problem.field_list:
    field_to_marker_dict[field.name] = patches.Circle((field.location.X, field.location.Y), radius=20, color='green')
    ax.add_patch(field_to_marker_dict[field.name])

for barn in problem.barn_list:
    circle = patches.Circle((barn.location.X, barn.location.Y), radius=20, color='black')
    ax.add_patch(circle)

for tractor in problem.tractor_list:
    verts = [(tractor.barn.location.X, tractor.barn.location.Y)]

    codes = [
     Path.MOVETO
    ]
    path = Path(verts, codes)

    color = generate_color()

    tractor_to_route_color_dict[tractor.name] = color

    tractor_to_route_dict[tractor.name] = patches.PathPatch(path, facecolor=color, lw=2)
    ax.add_patch(tractor_to_route_dict[tractor.name])


def update_routes(solution):
    for tractor in solution.tractor_list:
        # Select the current route
        route = tractor_to_route_dict[tractor.name]

        # Change color of all fields belonging to this route
        for field in tractor.field_list:
            field_to_marker_dict[field.name].color = tractor_to_route_color_dict[tractor.name]

        # Set the first point for tractor at the starting barn point
        verts = [(tractor.barn.location.X,tractor.barn.location.Y)]
        codes = [Path.MOVETO]

        # Add the other points to the path
        verts.extend(map(lambda field: (field.location.X,field.location.Y), tractor.field_list))
        for i in range(len(verts)-1):
            codes.append(Path.LINETO)

        # Close the loop, back to the barn
        # locations.append(locations[0])
        verts.append(verts[0])
        codes.append(Path.LINETO)

        path = Path(verts, codes)

        route = patches.PathPatch(path, edgecolor=tractor_to_route_color_dict[tractor.name], facecolor="none", lw=2)
        ax.add_patch(route)
        fig.canvas.draw()


update_routes(final_solution)
ax.axis('equal')
plt.show()

## And we can see how the solution is represented

This is a list of the tractors, identified by their name (number), and the array of locations that must go through, respecting our constraints.

In [ ]:
for tractor in final_solution.tractor_list:
    # Select the current route
    route = tractor_to_route_dict[tractor.name]

    # Change color of all fields belonging to this route
    for field in tractor.field_list:
        field_to_marker_dict[field.name].color = tractor_to_route_color_dict[tractor.name]

    verts = [(tractor.barn.location.X, tractor.barn.location.Y)]
    verts.extend(map(lambda field: (field.location.X, field.location.Y), tractor.field_list))

    print(tractor.name)
    print(verts)